In [ ]:
from dotenv import load_dotenv
import requests
import numpy as np
import pandas as pd
import os
import time

In [ ]:
load_dotenv() 
THE_MOVIE_DB_API_KEY = os.getenv("THE_MOVIE_DB_API_KEY")
THE_MOVIE_DB_ACCESS_TOKEN = os.getenv("THE_MOVIE_DB_ACCESS_TOKEN")
THE_MOVIE_DB_BASE_URL = os.getenv("THE_MOVIE_DB_BASE_URL")

In [4]:
headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {THE_MOVIE_DB_ACCESS_TOKEN}"
}

In [6]:
#initial crawl
def get_movies(num_pages: int, max_retries: int = 5) -> pd.DataFrame:

    entries = []

    url = THE_MOVIE_DB_BASE_URL + "/discover/movie?include_adult=false&include_video=false&language=en-US&page={page}&sort_by=popularity.desc" #type: ignore
    for page in range(1, num_pages + 1):
        for _ in range(max_retries):
            response = requests.get(url.format(page=page), headers=headers)
            if response.status_code == 200:
                break
            time.sleep(1)
        else:
            raise Exception("Failed to get data after maximum retries")
        
        data = response.json()["results"]

        entries.extend(data)

    return pd.DataFrame(entries)

In [ ]:
raw_data = get_movies(30)

In [8]:
raw_data.to_csv("raw_data.csv", index=False)